In [188]:
from random import randint, sample, random, seed
from itertools import chain
from scipy import sparse
from scipy.special import expit
import numpy as np
from pprint import PrettyPrinter
import matplotlib.pyplot as plt
from numpy.random import choice
from numpy import array
from math import floor, ceil

%matplotlib inline

print = PrettyPrinter(indent=4).pprint

In [187]:

def _create_conections(n_nodes=10, ):


    test = sparse.lil_matrix((n_nodes, n_nodes))

    indexes = np.array(sample(range(n_nodes), 5))

    test[0, 1] = 1

    print(vec)

    out = test.dot(vec)

print(out)

NameError: name 'out' is not defined

In [ ]:
my_dict = {'test': 5}
ret = my_dict.get('other')
print(ret)

In [172]:
seed(1)
sample(range(50), 3)

[8, 36, 48]

In [ ]:
%whos

In [ ]:


mean = 4.4
n_projecting_nodes = 1000
low = int(floor(mean))
high = int(ceil(mean))
pl = high - mean
ph = 1 - pl

arr = array(choice([low, high], n_projecting_nodes, p=[pl, ph]), dtype=int)
np.sum(arr)/n_projecting_nodes

In [180]:
_INITIAL_UNITS = 2

_INITIAL_MIN_NODES = 8

_INITIAL_MAX_NODES = 14

_INITIAL_LAYERS_MIN = 3

_INITIAL_LAYERS_MAX = 5

_INITIAL_MU_RANGE = 0.5

_INITIAL_SIGMA = 0.3

_INITIAL_OUTDEGREE_MIN = 0

_INITIAL_OUTDEGREE_MAX = 4

_INITIAL_PROJECTIONS_PER_LAYER = 2

_N_INPUT_UNITS = 14

_N_OUTPUT_UNITS = 5


def _rand_range(rng):
    return random() * 2 * rng - rng


def _create_initial_structure():
    """Create the inital randomized structure for the agent.

    For the structure, see the ANNStructuredAgent configuration below.

    """
    seed(11)
    # Create units (populations or columns) and layers with their nodes count.
    structure = [
        [(randint(_INITIAL_MIN_NODES, _INITIAL_MAX_NODES), {})
            for _ in range(randint(_INITIAL_LAYERS_MIN, _INITIAL_LAYERS_MAX))]
        for _ in range(_INITIAL_UNITS)
    ]
    
    structure[0][0] = (_N_INPUT_UNITS, {})    
    structure[-1][-1] = (_N_OUTPUT_UNITS, {})
    
    print(structure)
    
    nodes = list(chain.from_iterable([[i[0] for i in j] for j in structure]))
    n_layers = len(nodes)
    n_nodes = sum(nodes)
    print(n_nodes)

    
    # For each layer in each unit: add the configuration (connections).
    
    for j, unit in enumerate(structure):
        for i, (_, conns) in enumerate(unit):
            # Connect this layer to other layers in the same unit.
            others = sample([n for n in range(len(unit)) if n != 0], _INITIAL_PROJECTIONS_PER_LAYER)
            for o in others:
                conns[o] = (
                    _rand_range(_INITIAL_MU_RANGE), _INITIAL_SIGMA,
                    randint(_INITIAL_OUTDEGREE_MIN, _INITIAL_OUTDEGREE_MAX))
            # Connect the last layer of the current unit to other units.
            if i == len(unit) - 1 and j < len(structure) - 1:
                others = [n for n in range(len(structure)) if n != j]
                conns[None] = {
                    n: (abs(_rand_range(_INITIAL_MU_RANGE)), _INITIAL_SIGMA,
                        randint(
                            _INITIAL_OUTDEGREE_MIN, _INITIAL_OUTDEGREE_MAX))
                    for n in others}
    # TODO: set nodes count for global input and output manually.
    return structure, nodes, n_nodes, n_layers

structure, nodes, n_nodes, n_layers = _create_initial_structure()





print(structure)

[   [(14, {}), (12, {}), (14, {}), (14, {})],
    [(11, {}), (12, {}), (14, {}), (5, {})]]
96
[   [   (   14,
            {   1: (0.30390031272104856, 0.3, 3),
                3: (0.12988272021680192, 0.3, 1)}),
        (   12,
            {   1: (-0.1965987373754745, 0.3, 0),
                2: (0.038693508577678504, 0.3, 0)}),
        (   14,
            {   2: (0.15007097391407254, 0.3, 4),
                3: (0.46475778112556676, 0.3, 4)}),
        (   14,
            {   None: {1: (0.46991741077521143, 0.3, 3)},
                1: (-0.440448894831145, 0.3, 1),
                3: (0.37956497253435106, 0.3, 4)})],
    [   (   11,
            {   2: (0.09095944346805751, 0.3, 1),
                3: (0.019124114764076716, 0.3, 2)}),
        (   12,
            {   1: (0.15480163900675548, 0.3, 3),
                2: (0.16244953189036815, 0.3, 3)}),
        (   14,
            {   1: (0.2579900490467689, 0.3, 4),
                3: (0.20780962149794946, 0.3, 2)}),
        (   5,
      

In [ ]:
def foo(result, i):
    i += 1
    print((i, result))
    if i == 50: 
        return result
    else: 
        foo(expit(result), i)

foo(0, 0)
1-1/np.e

In [ ]:
x = np.linspace(-10,10,100)

figure(figsize=(19,9))
plot(x, expit((x)*5-5))